В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск.

In [2]:
import numpy as np
import pandas as pd
import sys
from sklearn import svm, grid_search, datasets
from sklearn.cross_validation import KFold

Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [ ]:
main_data = pd.read_csv('data-logistic.csv')
main_data.columns=["A","B","C"];
X_data = main_data[['B','C']]
Y_data = main_data['A']
type(Y_data)
print X_data.info
print Y_data.size

<bound method DataFrame.info of              B          C
0     1.994596   2.468025
1    -1.247395   0.749425
2     2.309374   1.899836
3     0.849143   2.407750
4     1.454271  -0.665416
5     2.254227   2.263786
6    -0.067580   1.469141
7    -0.861961  -0.824856
8     0.699179   2.032488
9    -0.161480   0.495462
10   -0.147102   0.419647
11    0.080296  -1.156097
12    1.720655  -1.080662
13    1.573958   1.785434
14    1.168445  -0.645619
15   -0.066627  -1.059747
16    1.163355   1.745887
17    2.096970   1.129638
18    3.035705   2.142838
19    0.831852  -0.606777
20  -12.977800  -4.573056
21   -0.842603  -1.076282
22   -0.473537  -0.734473
23    1.540373   0.167921
24    0.495341   0.066751
25    2.091486   2.550005
26    2.937055   3.260800
27   -0.501667   0.318889
28    0.830207   1.234621
29   -0.390148  -0.436249
..         ...        ...
174   0.212184   2.701000
175  -0.239311   1.362500
176   1.828638   1.608036
177   0.277534  -0.758549
178   2.586595   2.593424
179   

In [ ]:
main_data2 = pd.read_csv('data-logistic_with_header.csv')
X_data = main_data[['B','C']]
Y_data = main_data['A']

print X_data.info
print Y_data.size



<bound method DataFrame.info of              B          C
0     1.994596   2.468025
1    -1.247395   0.749425
2     2.309374   1.899836
3     0.849143   2.407750
4     1.454271  -0.665416
5     2.254227   2.263786
6    -0.067580   1.469141
7    -0.861961  -0.824856
8     0.699179   2.032488
9    -0.161480   0.495462
10   -0.147102   0.419647
11    0.080296  -1.156097
12    1.720655  -1.080662
13    1.573958   1.785434
14    1.168445  -0.645619
15   -0.066627  -1.059747
16    1.163355   1.745887
17    2.096970   1.129638
18    3.035705   2.142838
19    0.831852  -0.606777
20  -12.977800  -4.573056
21   -0.842603  -1.076282
22   -0.473537  -0.734473
23    1.540373   0.167921
24    0.495341   0.066751
25    2.091486   2.550005
26    2.937055   3.260800
27   -0.501667   0.318889
28    0.830207   1.234621
29   -0.390148  -0.436249
..         ...        ...
174   0.212184   2.701000
175  -0.239311   1.362500
176   1.828638   1.608036
177   0.277534  -0.758549
178   2.586595   2.593424
179   

В качестве метрики качества будем использовать AUC-ROC (Area Under ROC-Curve).
В Scikit-Learn метрика AUC реализована функцией sklearn.metrics.roc_auc_score. В качестве первого аргумента ей передается вектор истинных ответов,
в качестве второго — вектор с оценками принадлежности объектов к
первому классу.

http://datareview.info/article/sovremennyie-metodyi-analiza-tonalnosti-teksta/


In [123]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l2',random_state=242)
print lr
lr.fit(X_data, Y_data)

print 'Test Accuracy: %.2f'%lr.score(X_data, Y_data)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=242, shuffle=True,
       verbose=0, warm_start=False)
Test Accuracy: 0.82


In [124]:
#Create ROC curve
from sklearn.metrics import roc_auc_score, auc
import matplotlib.pyplot as plt

pred_probas = lr.predict_proba(X_data)[:,1]  # returns (205,)  of (205, 2)
rez = roc_auc_score(Y_data, pred_probas)
print rez
# 0.910485810486 _with no headers
# 0.910485810486 _with_headers

0.910485810486


In [ ]:
# http://pastebin.com/3Qepftnt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
 
data = pd.read_csv('data-logistic.csv', header=None)
y = np.array(data.pop(0))
X = np.array(data)
#print X
 
w = np.zeros(2)
k = 0.1
C = 10.0
delta = 0.1
 
while delta > 0.00001 :
    s = [0, 0]
    for i in xrange(len(y)) :
        s = s + X[i]*y[i]*(1.0-1.0/(1.0+np.exp(-1.0*y[i]*np.sum(w*X[i]))))
    w_prev = w
    w = w + s*k/len(y) - k*C*w
    delta = np.sqrt(np.sum((w-w_prev)**2))
 
a = np.zeros_like(y)
for i in xrange(len(y)) :
    a[i] = 1.0/(1.0+np.exp(-1.0*np.sum(w*X[i])))
 
print w, C, roc_auc_score(y, a)

[ 0.02855875  0.02478014] 10.0 0.5


In [ ]:
from pandas import read_csv
from math import exp
from numpy.numarray import dot
from math import sqrt, pow
 
data = read_csv('data-logistic.csv', header=None)
 
y = data[0]
X = data.iloc[:, 1:]
l = len(X)
w = (0, 0)
k = 0.1
border = 1e-5
C = 10
 
print zip(X,y)
 
#def fn(y, x, w, i)
 
def update_weight(X, Y, w, i):
        return w[i] + k/l*sum([(Y[j]*X[j][i]*(1-1/(1+exp((-Y[j])*dot(X[j], w))))-k*C*w[i]) for j in xrange(len(Y))])
 
def distance(w1, w2):
    return sqrt(pow(w2[0]-w1[0], 2) + pow(w2[1]-w1[1], 2))
 
for j in xrange(10000):
    new_w = (update_weight(X, y, w, 0), update_weight(X, y, w, 1))
    dist = distance(w, new_w)
    if dist < border:
        break
    w = new_w
 
print w

ImportError: No module named numarray

In [ ]:
W1 = w1+k*1/L*np.sum([Y[i]*X1[i]*(1-1/1+math.exp(-Y[i]*(w1*X1[i]+w2*X2[i]))) for i in range(Y.size) ] )-k*C*w1
W2= w2+k*1/L*np.sum([Y[i]*X2[i]*(1-1/1+math.exp(-Y[i]*(w1*X1[i]+w2*X2[i]))) for i in range(Y.size) ] )-k*C*w2

In [ ]:
# http://pastebin.com/kMEYimti
import pandas
def lab_7():
    from sklearn.linear_model import LogisticRegression
 
    from sklearn.metrics import roc_auc_score
 
    data = pandas.read_csv('data-logistic.csv')
    y = np.array(data.icol(0))
    X = np.array(data.icol([1,2]))
 
    def sygmoid(x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))
 
    def gradientDescent(X, y, theta, alpha, numIterations, step):
        m, n = np.shape(X)
        pos = 1
        stop = numIterations
        last_theta = theta
 
        while pos < stop:
            predicted_values = np.dot(X, theta)
            theta = theta + alpha/m * np.dot((y - predicted_values), X)
            euc_dist = np.linalg.norm(theta-last_theta)
 
            if abs(euc_dist) > alpha:
                print 'Converged, iterations: %s !!!' % pos
                return last_theta
            else:
                last_theta = theta
                pos += step
 
        print 'Max iterations reached: %s !!!' % pos, theta
 
        return theta
 
    theta = np.array((0,0))
    alpha = 1e-5
    iterations = 10000
    step = 0.1
 
    gd = np.array(gradientDescent(sygmoid(X), sygmoid(y), theta, alpha, iterations, step))
 
    weights = {1 : gd[0], -1 : gd[1]}
 
    clf1 = LogisticRegression(penalty='l1', verbose=True, class_weight=weights)
    clf2 = LogisticRegression(penalty='l2', verbose=True, C=10, class_weight=weights)
 
    reg1 = clf1.fit(X, y)
    reg2 = clf2.fit(X, y)
 
    pred1 = reg1.predict(sygmoid(X))
    pred2 = reg2.predict(sygmoid(X))
    # pred1 = reg1.predict(X)   # my 
    # pred2 = reg2.predict(X)   # My


    #    print pred1, sygmoid(pred1)

    score1 = roc_auc_score(y, sygmoid(pred1))
    score2 = roc_auc_score(y, sygmoid(pred2))
 
 
    print "=====",gd, score1, score2
lab_7()

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]


Max iterations reached: 10000.0 !!! [ 0.23734769  0.23390218]
[LibLinear][LibLinear]===== [ 0.23734769  0.23390218] 0.693362193362 0.889321789322


# Это фрагмент использованный для решения - все остальные лабуда

In [ ]:
# http://pastebin.com/Cq5ywjri
from math import sqrt
import math
import pandas
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
 
 
def sigmoid(x1, x2, _w1, _w2):
    return 1 / (1 + math.exp(-_w1 * x1 - _w2 * x2))
 
 
def sigmoid_from_series(_X, _w1, _w2):
    list = []
    X1_list = _X[1].tolist()
    X2_list = _X[2].tolist()
    length = len(X1_list)
    for i in range(0, length):
        list.append(sigmoid(X1_list[i], X2_list[i], _w1, _w2))
    return list
 
 
def find_w(param, _w1, _w2, _l, _X, _Y, _C):
    k = 0.1
    if param == 'First':
        w_actual = _w1
        X_list = _X[1].tolist()
    else:
        if param == 'Second':
            w_actual = _w2
            X_list = _X[2].tolist()
        else:
            raise ValueError
    Y_list = _Y.tolist()
    X1_list = _X[1].tolist()
    X2_list = _X[2].tolist()
    sum = 0
    for i in range(0, _l - 1):
        sum += Y_list[i] * X_list[i] * (1 - 1 / (1 + math.exp(-Y_list[i] * (_w1 * X1_list[i] + _w2 * X2_list[i]))))
 
    return w_actual + k * sum / l - k * C * w_actual
 
 
if __name__ == '__main__':
    data = pandas.read_csv('data-logistic.csv', header=None)
    Y = data[0]
    X = data[[1, 2]]
    l = len(Y)
    # print(l)
    e = 100
    num_it = 0
    w1 = 0
    w2 = 0
    C = 10  # first starting with C=0, res=0.928, second starting C=10, res=0.936
    while e > 0.00001 and num_it < 10000:
        num_it += 1
        # Save previous w1 , w2
        w1_prev = w1
        w2_prev = w2
 
        # find w1 , w2
        w1 = find_w('First', w1_prev, w2_prev, l, X, Y, C)
        w2 = find_w('Second', w1_prev, w2_prev, l, X, Y, C)
        e = sqrt((w1 - w1_prev) ** 2 + (w2 - w2_prev) ** 2)
    print(num_it)
    print(w1)
    print(w2)
    # С=0 242  0.284232310254   0.0973847779184
    # C=10 8   0.0285402045405 0.024965725476
    res = roc_auc_score(Y.tolist(), sigmoid_from_series(X, w1, w2))
    print(res)


8
0.0285402045405
0.024965725476
0.936095238095
